# LangChain 实战：电器销售聊天机器人

In [1]:
with open("electrical_sales_data.txt") as f:
    electrical_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。', metadata={})

In [6]:
len(docs)

70

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [8]:
query = "这个电器怎么用"

In [9]:
answer_list = db.similarity_search(query)

In [ ]:
for ans in answer_list:
    print(ans.page_content + "\n")

In [11]:
db.save_local("electrical_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f848fb86560>, search_type='similarity', search_kwargs={'k': 3})

In [ ]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

In [15]:
docs = topK_retriever.get_relevant_documents("你们有没有扫地机器人？")

In [ ]:
for doc in docs:
    print(doc.page_content + "\n")

#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [ ]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [ ]:
docs[0].page_content

In [ ]:
docs[0].page_content.split("[销售回答] ")

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [ ]:
ans

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [ ]:
query = "我想买台电视"

print(sales(query))

In [ ]:
print(sales(query, 0.75))

In [ ]:
query = "价格1万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": query})

In [ ]:
qa_chain({"query": "小区吵不吵"})

In [ ]:
print(sales("小区吵不吵"))

## 加载 FAISS 向量数据库已有结果

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("electrical_sale", OpenAIEmbeddings())

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "我想买电脑，你们有么"})

In [33]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [ ]:
qa_chain({"query": "我想买电脑，你们有么"})

In [35]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买电脑，你们有么"})

In [ ]:
result